# Building an AI Book Research Assistant

This notebook provides step-by-step instructions for replicating what Andrew did in the second demo. Please follow along, and feel free to play with different variations too! At the end, there are optional instructions for building a more advanced book research assistant than Andrew showed. 

*Useful Resources*:

- **AISuite GitHub Repo**: [https://github.com/andrewyng/aisuite](https://github.com/andrewyng/aisuite)
- **Open Library API**:
  - [Open Library](https://openlibrary.org/developers/api) offers free and public Web APIs for accessing book and author catalog data (no API key is required)
  - See `openlibrary_api_docs.md` in this lesson's directory for more information

## Step 1: Use Open Library API's Documentation

- Open Jupyter chat by clicking on the chat bubble icon on the left sidebar of Jupyter Lab
- In the Chat interface, create a new chat by clicking `+Chat` (choose any name for the file)
- In the chat window, attach the Markdown file `openlibrary_api_docs.md` using the `@` symbol:
  - Type `@`, then select `file` from the autocomplete menu
  - You'll see a list of available files in the lesson's directory, choose `openlibrary_api_docs.md` 
- Use a prompt like this:
   > Write a code cell that uses Open Library's search endpoint to look up books based on a user's query and returns a response listing identified books, with these fields for each book: key, title, author_name, first_publish_year, isbn, cover_id, edition_count, language, subject. Use a variable called limit, set to 5, to specify the number of returned results. Print the results. Use the Open Library API documentation in the attached file.
- Transfer the generated code to the cell below and run it

In [4]:
import requests

# User-defined query and number of results to return
query = input("Enter a search query: ").strip()
limit = 5

# Define the fields to retrieve
fields = "key,title,author_name,first_publish_year,isbn,cover_i,edition_count,language,subject"

# Perform the API request
response = requests.get(
    "https://openlibrary.org/search.json",
    params={
        "q": query,
        "fields": fields,
        "limit": limit
    }
)

# Parse JSON response
data = response.json()

# Display results
docs = data.get("docs", [])
print(f"\nFound {len(docs)} results (showing up to {limit}):\n")

for book in docs:
    print({
        "key": book.get("key"),
        "title": book.get("title"),
        "author_name": book.get("author_name"),
        "first_publish_year": book.get("first_publish_year"),
        "isbn": book.get("isbn"),
        "cover_id": book.get("cover_i"),
        "edition_count": book.get("edition_count"),
        "language": book.get("language"),
        "subject": book.get("subject")
    })
    print("-" * 80)

Enter a search query:  pega



Found 5 results (showing up to 5):

{'key': '/works/OL103123W', 'title': 'Fahrenheit 451', 'author_name': ['Ray Bradbury'], 'first_publish_year': 1953, 'isbn': ['1455106321', '9780557077649', '9781530031832', '0552076546', '9781951151232', '9783257261042', '9780007491568', '0345250494', '9787532757763', '3453319834', '9788466345392', '0307347974', '8525052248', '9781514801604', '9863442798', '9780246635716', '9781644730539', '8496822966', '8445006770', '9780812415551', '9780783883137', '9783257071405', '1613832494', '0786175370', '0694526266', '9789863442790', '3453164121', '9785699455379', '9786558300151', '9788445071168', '9781733075862', '9783453164123', '1530031680', '9788418252600', '8490321477', '1451690312', '9789875661196', '9788484501961', '1491536357', '9780345342966', '9788674360729', '0812973577', '0557077648', '9789506440299', '8804320257', '067187229X', '9788401422829', '7536671490', '9788445071199', '1484475739', '9780552076548', '9780606252560', '3257229046', '97884758

## Step 2: Wrap the API Call in a Tool Function

- In the chat interface, use a prompt like this:
  > Wrap the code above in a function `search_books`, with good docstring.
  > 
  > - Input arguments: `query` and `limit` (with default value of 10)
  > - Output: JSON string with:
  >   - `num_found`: total results
  >   - `books`: list of dicts with title, authors, first_publish_year, key, edition_count

In [10]:
import requests
import json

def search_books(query: str, limit: int = 10) -> str:
    """
    Search for books using the Open Library Search API.

    Parameters
    ----------
    query : str
        The keyword or phrase to search for (e.g., title, author, or subject).
    limit : int, optional
        The maximum number of book results to return (default is 10).

    Returns
    -------
    str
        A JSON-formatted string containing:
        - num_found: the total number of matching results from Open Library
        - books: a list of dictionaries with the following fields for each book:
            * title
            * authors
            * first_publish_year
            * key
            * edition_count
    """
    fields = "key,title,author_name,first_publish_year,edition_count"

    response = requests.get(
        "https://openlibrary.org/search.json",
        params={
            "q": query,
            "fields": fields,
            "limit": limit
        }
    )
    response.raise_for_status()
    data = response.json()

    books = []
    for doc in data.get("docs", []):
        books.append({
            "title": doc.get("title"),
            "authors": doc.get("author_name"),
            "first_publish_year": doc.get("first_publish_year"),
            "key": doc.get("key"),
            "edition_count": doc.get("edition_count")
        })
        print(books)

    result = {
        "num_found": data.get("numFound", 0),
        "books": books
    }

    return json.dumps(result, indent=2)

# Example usage:
# print(search_books("Pride and Prejudice", limit=5))

- Test the function by uncommenting the code below:

In [11]:
results = search_books("dragon")

[{'title': 'The Book of Dragons', 'authors': ['Edith Nesbit'], 'first_publish_year': 1973, 'key': '/works/OL99529W', 'edition_count': 706}]
[{'title': 'The Book of Dragons', 'authors': ['Edith Nesbit'], 'first_publish_year': 1973, 'key': '/works/OL99529W', 'edition_count': 706}, {'title': 'The reluctant dragon', 'authors': ['Kenneth Grahame', 'Jim Weiss', 'Ernest H. Shepard', 'Michael Hague', 'Kenneth Grahame', 'Kenneth Kenneth Grahame', 'Grahame Kenneth'], 'first_publish_year': 1938, 'key': '/works/OL69602W', 'edition_count': 228}]
[{'title': 'The Book of Dragons', 'authors': ['Edith Nesbit'], 'first_publish_year': 1973, 'key': '/works/OL99529W', 'edition_count': 706}, {'title': 'The reluctant dragon', 'authors': ['Kenneth Grahame', 'Jim Weiss', 'Ernest H. Shepard', 'Michael Hague', 'Kenneth Grahame', 'Kenneth Kenneth Grahame', 'Grahame Kenneth'], 'first_publish_year': 1938, 'key': '/works/OL69602W', 'edition_count': 228}, {'title': 'The Eyes of the Dragon', 'authors': ['Stephen King'

## Step 3: Build a Chat Handler with Automatic Function Calling

- Drag and drop the raw cell below into the chat window and use a prompt like this:
  > Use this aisuite example to create a simple chat handler function `chat_with_tools`:
  > - input: `user_message`
  > - output: `response.choices[0].message.content`

In [12]:
import aisuite as ai

# Initialize the AISuite client
client = ai.Client()

def chat_with_tools(user_message: str) -> str:
    """
    Handle a user chat message and return the AI-generated response
    using tool-assisted reasoning.

    Parameters
    ----------
    user_message : str
        The text message or query from the user.

    Returns
    -------
    str
        The assistant’s message content from the AI model.
    """
    messages = [
        {
            "role": "user",
            "content": user_message
        }
    ]

    # Automatic tool execution with max_turns specified
    response = client.chat.completions.create(
        model="openai:gpt-4.1-mini",
        messages=messages,
        tools=[search_books],  # Add function tools here
        max_turns=3            # Maximum number of back-and-forth tool calls
    )

    return response.choices[0].message.content

# Example usage:
# reply = chat_with_tools("Suggest some books about artificial intelligence.")
# print(reply)

## Step 4: Test the Chat Handler

- Run the following code cell to test `chat_with_tools`
- Feel free to search for different types of books
- Optional: If your generated function `search_books` does not contain printing statements, you can update the function as Andrew showed in the video:
  - prompt used: modify the search_books function to add a printing statement showing when it is called with basic status messages.

In [13]:
print("="*60)
print("EXAMPLE: Science Function book search")
print("="*60)

user_query = "Search for books about science fiction"
print(f"\n👤 User: {user_query}")

response = chat_with_tools(user_query)

print(f"\n🤖 Assistant: {response}")

EXAMPLE: Science Function book search

👤 User: Search for books about science fiction
[{'title': 'The Science Fiction Hall of Fame -- Volume One', 'authors': ['Robert Silverberg', 'Robert A. Heinlein', 'Arthur C. Clarke', 'Isaac Asimov', 'Ray Bradbury'], 'first_publish_year': 1970, 'key': '/works/OL1960472W', 'edition_count': 31}]
[{'title': 'The Science Fiction Hall of Fame -- Volume One', 'authors': ['Robert Silverberg', 'Robert A. Heinlein', 'Arthur C. Clarke', 'Isaac Asimov', 'Ray Bradbury'], 'first_publish_year': 1970, 'key': '/works/OL1960472W', 'edition_count': 31}, {'title': 'Foundation', 'authors': ['Isaac Asimov'], 'first_publish_year': 1951, 'key': '/works/OL46125W', 'edition_count': 97}]
[{'title': 'The Science Fiction Hall of Fame -- Volume One', 'authors': ['Robert Silverberg', 'Robert A. Heinlein', 'Arthur C. Clarke', 'Isaac Asimov', 'Ray Bradbury'], 'first_publish_year': 1970, 'key': '/works/OL1960472W', 'edition_count': 31}, {'title': 'Foundation', 'authors': ['Isaac A

## Step 5: Look up Details on Books (optional) 

You can repeat steps 1-3 to define a second tool for the book research assistant, so it can handle more complex book queries.
For example, you can use the `works` endpoint to retrieve more detailed information about a particular book given its work id. For more details, check `openlibrary_api_docs.md`.

Here are steps with suggested prompts:

1. In the chat interface, attach the file containing the API documentation of Open Library (`openlibrary_api_docs.md`) and use a prompt like this:
   > Write a code cell that uses Open Library's works endpoint to retrieve information about a book, based on the work's key that has this format: "/works/{work_id}". The result should include: title, description, subjects (limit to 10), first_sentence, covers (limit to 3). Print the returned results. Use the documentation for Open Library's API provided above.
2. In the chat interface, create the function tool using a prompt like this:
   > Wrap the code above in a function `get_book_details`:
   > - Function argument: `work_key` in this format: `/works/{work_id}`
   > - Output: JSON string with: title, description, subjects (limit to 10), first_sentence, covers (limit to 3)
   >  - Write good docstring
3. Update the `chat_with_tools` function to include the second tool as follows:
   
   ``` python
   response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books, get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
   )
   ```
   
4. Try this query:
   ```python
   user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"
   ```

In [15]:
import requests
import json

def get_work_details(work_key: str) -> dict:
    """
    Retrieve detailed information about a book from the Open Library Works API.

    Parameters
    ----------
    work_key : str
        The Open Library work key, e.g. "/works/OL45883W".

    Returns
    -------
    dict
        A dictionary containing:
        - title
        - description (string if available)
        - subjects (list of up to 10 strings)
        - first_sentence (string if available)
        - covers (list of up to 3 cover image IDs)
    """
    # Ensure the work key starts with '/works/'
    if not work_key.startswith("/works/"):
        raise ValueError("Invalid work key format. Expected format: '/works/{work_id}'")

    url = f"https://openlibrary.org{work_key}.json"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Normalize optional fields
    title = data.get("title")
    covers = data.get("covers", [])[:3]
    subjects = data.get("subjects", [])[:10]

    # Description and first_sentence can be nested or different types
    description = None
    if isinstance(data.get("description"), dict):
        description = data["description"].get("value")
    elif isinstance(data.get("description"), str):
        description = data["description"]

    first_sentence = None
    if isinstance(data.get("first_sentence"), dict):
        first_sentence = data["first_sentence"].get("value")
    elif isinstance(data.get("first_sentence"), str):
        first_sentence = data["first_sentence"]

    result = {
        "title": title,
        "description": description,
        "subjects": subjects,
        "first_sentence": first_sentence,
        "covers": covers
    }

    print(json.dumps(result, indent=2))
    return result

# Example usage:
# get_work_details("/works/OL45883W")

In [16]:
import requests
import json

def get_book_details(work_key: str) -> str:
    """
    Retrieve detailed information about a book from the Open Library Works API.

    This function queries the Open Library's `/works/{work_id}.json` endpoint
    to obtain structured metadata about a specific book (or work). The returned
    JSON string contains selected fields of interest such as the title,
    description, subjects, first sentence, and cover image identifiers.

    Parameters
    ----------
    work_key : str
        The Open Library work key, in this format: "/works/{work_id}".
        Example: "/works/OL45883W"

    Returns
    -------
    str
        A JSON-formatted string containing the following keys:
        - "title" : str — The book’s title.
        - "description" : str or None — The book’s description text, if available.
        - "subjects" : list[str] — Up to 10 subject names (topics, genres, etc.).
        - "first_sentence" : str or None — The first line of the book, if provided.
        - "covers" : list[int] — Up to 3 cover image IDs associated with this work.

    Raises
    ------
    ValueError
        If the provided `work_key` does not start with "/works/".
    requests.HTTPError
        If the HTTP request to the Open Library API fails.
    """
    # Validate work key format
    if not work_key.startswith("/works/"):
        raise ValueError("Invalid work key format. Expected format: '/works/{work_id}'")

    # Build API URL
    url = f"https://openlibrary.org{work_key}.json"

    # Perform HTTP request
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # Extract and normalize fields
    title = data.get("title")
    covers = data.get("covers", [])[:3]
    subjects = data.get("subjects", [])[:10]

    # Handle flexible description field
    description = None
    if isinstance(data.get("description"), dict):
        description = data["description"].get("value")
    elif isinstance(data.get("description"), str):
        description = data["description"]

    # Handle flexible first_sentence field
    first_sentence = None
    if isinstance(data.get("first_sentence"), dict):
        first_sentence = data["first_sentence"].get("value")
    elif isinstance(data.get("first_sentence"), str):
        first_sentence = data["first_sentence"]

    # Create result dictionary
    result = {
        "title": title,
        "description": description,
        "subjects": subjects,
        "first_sentence": first_sentence,
        "covers": covers
    }

    # Return result as a formatted JSON string
    return json.dumps(result, indent=2)

# Example usage:
# print(get_book_details("/works/OL45883W"))

In [18]:
import aisuite as ai
client = ai.Client()

user_query = "Search for books with 'Linear Algebra' in the title and tell me about the one with the most editions"

messages = [{
    "role": "user",
    "content": user_query
}]

# Automatic tool execution with max_turns
response = client.chat.completions.create(
    model="openai:gpt-4.1-mini",
    messages=messages,
    tools=[search_books,get_book_details],
    max_turns=3  # Maximum number of back-and-forth tool calls
)
print(response.choices[0].message.content)

[{'title': 'Elementary linear algebra', 'authors': ['Howard Anton', 'Chris Rorres'], 'first_publish_year': 1973, 'key': '/works/OL485862W', 'edition_count': 37}]
[{'title': 'Elementary linear algebra', 'authors': ['Howard Anton', 'Chris Rorres'], 'first_publish_year': 1973, 'key': '/works/OL485862W', 'edition_count': 37}, {'title': 'Linear algebra', 'authors': ['Stephen H. Friedberg', 'Arnold J. Insel', 'Lawrence E. Spence'], 'first_publish_year': 1979, 'key': '/works/OL3281311W', 'edition_count': 6}]
[{'title': 'Elementary linear algebra', 'authors': ['Howard Anton', 'Chris Rorres'], 'first_publish_year': 1973, 'key': '/works/OL485862W', 'edition_count': 37}, {'title': 'Linear algebra', 'authors': ['Stephen H. Friedberg', 'Arnold J. Insel', 'Lawrence E. Spence'], 'first_publish_year': 1979, 'key': '/works/OL3281311W', 'edition_count': 6}, {'title': 'Linear Algebra with Applications', 'authors': ['Gareth Williams'], 'first_publish_year': 1984, 'key': '/works/OL258590W', 'edition_count'